In [1]:
import sys
sys.path.append("../../")
import tensorflow as tf
import numpy as np
from src.basenet import BaseNetCompiler, KERAS_LIST_LAYERS, KERAS_LOSSES, KERAS_OPTIMIZERS
from IPython.display import display

# BaseNetCompiler

## Advanced use tutorial

In this JuPyter Notebook we will learn further uses and specifications of the ``BaseNetCompiler`` Class.

### Contents

1. About ``BaseNetCompiler``.
2. Construction.
    1. From code.
    2. Adding and substracting layers.
    3. Build From YAML files.
    4. Compiling the model.
3. Save and load compilers.
    1. Export the compiler to a YAML file.
4. Setting up the computational devices.
5. Visualize the available options (losses, optimizers and layers).


## 1. About BaseNetCompiler

* The BaseNetCompiler is an Python Class that contains the relevant information for the API to build a Machine Learning Model.
* The BaseNetCompiler is a custom compiler that takes the information about the network and compiles it with the given parameters.
* This API makes use of the Keras API and TensorFlow framework to build the AI models, so some Keras and Tensorflow knowledge is required (at least at high level).
* The BaseNetCompiler also allows the user to use a ``.yaml`` file to build the network with the following format:

        compiler:
          name: <name of the model>
          input_shape:
            - <input shape of the model (I)>
            - <input shape of the model (II)>
            - <...>
          output_shape: <output shape of the model>

          compile_options:
            loss: <tf.keras loss function name>
            optimizer: <tf.keras optimizer name>
            metrics:
              - <tf.keras loss function name provided as a loss function>
              - <'accuracy' is always a good metric to analyze>

          devices:
            - <your device type>:
                name: <the name of your device in BaseNetCompiler.show_devs()>
                state: <'Idle' for nothing, 'Train' for training>

            <some device examples:>
            - cpu:
                name: "/device:CPU:0"
                state: "Idle"
            - gpu:
                name: "/device:GPU:0"
                state: "Train"
            - gpu:
                name: "/device:GPU:1"
                state: "Idle"
            - gpu:
                name: "/device:GPU:n"
                state: "Idle"

          layers:
            - layer:
                name: <layer name in tf.keras.layers>
                shape:
                    - <layer shape (I)>
                    - <layer shape (II)>
                    - <...>
                options:
                    - option:
                        name: <the name of the option in tf.keras.layers.<your layer name> or
                               "{open}/{close}_pipeline">
                        value: <the value of the option in tf.keras.layers.<your layer name>>

            <some layer examples:>
            - layer:
                name: "Flatten"
                shape:
                options:

            - layer:
                name: "Dense"
                shape:
                  - 128
                options:
                  - option:
                      name: "activation"
                      value: "relu"

            - layer:
                name: "Dense"
                shape:
                  - 64
                options:

            - layer:
                name: "Dense"
                shape:
                  - 32
                options:
                  - option:
                      name: "activation"
                      value: "sigmoid"

            - layer:
                name: "open_pipeline"
                shape:
                options:

            - layer:
                name: "Dense"
                shape:
                  - 32
                options:
                  - option:
                      name: "activation"
                      value: "sigmoid"

            - layer:
                name: "open_pipeline"
                shape:
                options:

            - layer:
                name: "Dense"
                shape:
                  - 32
                options:
                  - option:
                      name: "activation"
                      value: "sigmoid"

            - layer:
                name: "close_pipeline"
                shape:
                options:

When open_pipeline is provided, the model creates a separate pipeline for the incoming layers. If more than one
open_pipeline is provided, more pipelines will be added. When close_pipeline is provided, a
``tf.keras.layers.Concatenate`` layer is added into the model to close all the previous models into the main pipeline.

This compiler implements some TensorFlow functions to list the GPU devices.
    
## 2. Construction

To build a BaseNetDatabase we usually give it two parameters: 'x' and 'y'. 'x' contains the input values of the model and 'y' contains the solutions.

        BaseNetCompiler(io_shape: tuple[int], compile_options: dict, devices: dict, layers: list[dict],
                        name: str, verbose: bool):
        
Build the BaseNetCompiler class.

        :io_shape: Input-output shape [(input,), output].
        :compile_options: Dictionary of compiling options {loss: , optimizer: , metrics: }.
        :devices: {device: role}. Consider calling: BaseNetCompiler.show_devs().
        :layers: List of layers: {name: ( (shape,) , {'args': args} )}.
        :name: Name of the model.
        :verbose: Print state and errors in the BaseNetCompiler.

In [2]:
# Definition:
def __init__(self, io_shape: tuple, compile_options: dict, devices: dict, layers: list[dict] = None,
             name: str = 'current_model', verbose: bool = False):
    [...]

#### 2.1. Construction from code.

The ``io_shape`` is always a tuple, even if the input is one-dimensional. The output of every Deep Learning model is one-dimensional, so you only need to specify the dimension as an integer. 

Let's build a CNN with the BaseNetCompiler with the following specifications:

* Input of a 256x256 image in gray scale (1 channel)
* A convolutional layer with zero padding.
* Max pooling.
* Other convolutional layer.
* Max pooling.
* Flatten layer.
* Dense layer.
* Output of 1 bit as a boolean.

See the variable ``KERAS_LIST_LAYERS`` to see the available layers for a BaseNetCompiler.


The network optimization will follow:

* Loss: MSE.
* Optimizer: Adaptative momentum.


Let's define the input-output shapes and the compile options as a dictionary containing 3 values:

* loss: Loss function (see the variable ``KERAS_LOSSES`` and refer to the Keras API for further documentation).
* optimizer: Optimization strategy (see the variable ``KERAS_OPTIMIZERS`` and refer to the Keras API for further documentation).
* metrics: The metrics you want to follow (None in this case, as loss is always provided).

In [3]:
input_shape = (256, 256, 1)
output_shape = 1
io_shape = (input_shape, output_shape)
compile_options = {'loss': 'mean_squared_error', 'optimizer': 'adam'}
print('Input-output shape:', io_shape)
print('Compile options:', compile_options)

Input-output shape: ((256, 256, 1), 1)
Compile options: {'loss': 'mean_squared_error', 'optimizer': 'adam'}


To see the training devices your machine has available for Python, you can call the method ``BaseNetCompiler.show_devs()``. In this case, we should only have a 1 CPU, automatically asigned for train.

This will be our distribution for training (CPU0 -> Train),  and we will store that into ``devices``. A device can have 2 states: ``Train`` and ``Idle``. We can switch the variables in the dictionary to make a custom set-up.

In [4]:
devices = BaseNetCompiler.show_devs()
devices

{'/device:CPU:0': 'Train'}

Now it is time to build the layers, it is a list of dictionaries:

* {layer_name: (layer_shape, {layer_arguments, layer_argument_values})}
* {'Dense': ((256,), {'activation': 'relu'})} -> This will create a 256 dimensional Dense layer with the activation function as ReLu.

According to the specifications:

In [5]:
layers = [
    {'Conv2D': ((32, 3), {'padding': 'same'})},
    {'MaxPooling2D': ((), {})},
    {'Conv2D': ((32, 3), {})},
    {'MaxPooling2D': ((), {})},
    {'Flatten': ((), {})},
    {'Dense': ((256,), {})}
]

It's time to build the BaseNetCompiler, where the ``verbose=True`` option if for console logging and check if the BaseNetCompiler is well build by accessing the ``is_valid`` attribute or by its boolean:

In [6]:
my_compiler = BaseNetCompiler(io_shape=io_shape,
                              compile_options=compile_options,
                              devices=devices,
                              layers=layers,
                              name='my_test_cnn',
                              verbose=True)
print('Is my compiler valid?:', my_compiler.is_valid)
if my_compiler:
    print(f'The compiler {my_compiler} is valid.')
else:
    print(f'The compiler {my_compiler} is NOT valid.')

Is my compiler valid?: True
The compiler Compiler with 6 layers, options:
{'loss': 'mean_squared_error', 'optimizer': 'adam'} is valid.


#### 2.2. Adding and substracting layers:
Other way to build the model is to add the layers step by step with the ``add`` method. Let's re-build the compiler with no layers:

In [7]:
my_compiler = BaseNetCompiler(io_shape=io_shape,
                              compile_options=compile_options,
                              devices=devices,
                              name='my_test_cnn',
                              verbose=True)
print('Is my compiler valid?:', my_compiler.is_valid)

Is my compiler valid?: False


Now we append the layers by the ``add`` method or the ``+`` operator:

* You can specify the place in the ``where`` attribute.
* By default, they are appended at the end of the list of layers.

In [8]:
my_compiler.add({'Conv2D': ((32, 3), {'padding': 'same'})})
my_compiler.add({'MaxPooling2D': ((), {})})
my_compiler.add({'Conv2D': ((32, 3), {})})

Compiler with 3 layers, options:
{'loss': 'mean_squared_error', 'optimizer': 'adam'}

In [9]:
my_compiler + {'MaxPooling2D': ((), {})}
my_compiler + {'Dense': ((256,), {})}
my_compiler.layers

[{'Conv2D': ((32, 3), {'padding': 'same'})},
 {'MaxPooling2D': ((), {})},
 {'Conv2D': ((32, 3), {})},
 {'MaxPooling2D': ((), {})},
 {'Dense': ((256,), {})}]

Now, we notice that everything is okay. However, we forgot to add the ``'Flatten'`` layer in position 4. So we can add it using the ``where`` option:

In [10]:
my_compiler.add({'Flatten': ((), {})}, where=4)
my_compiler.layers

[{'Conv2D': ((32, 3), {'padding': 'same'})},
 {'MaxPooling2D': ((), {})},
 {'Conv2D': ((32, 3), {})},
 {'MaxPooling2D': ((), {})},
 {'Flatten': ((), {})},
 {'Dense': ((256,), {})}]

We can also remove layers using the ``pop()`` method. That takes the index of the layer to be poped starting from the bottom of the list. So to extract the last layer we will use ``pop(0)``:

In [11]:
popped_layer = my_compiler.pop(0)
print('Popped layer: ', popped_layer)
my_compiler + popped_layer
my_compiler.layers

Popped layer:  {'Dense': ((256,), {})}


[{'Conv2D': ((32, 3), {'padding': 'same'})},
 {'MaxPooling2D': ((), {})},
 {'Conv2D': ((32, 3), {})},
 {'MaxPooling2D': ((), {})},
 {'Flatten': ((), {})},
 {'Dense': ((256,), {})}]

#### 2.3. Building from YAML files:

There is other way to build the models in a easy way. You can use a YAML to define your model easily. You can open the ``./compilers/example_cnn.yaml`` file and see how to build this same CNN with the YAML file.

There are 5 main attributes and all are compulsory inside the ``compiler`` attribute:

* **name**: The model name (string).
* **input_shape**: A list of shapes (integers).
* **output_shape**: A single value of output shape (integer).
* **devices**: A set of devices telling their state (train or idle).
* **layers**: A list of layers telling their shape and options.

Example of name attribute:

    name: "my_compiler1"
    
Example of input shape:

    input_shape:
        - 64
        - 64
        - 1
        
Example of output shape:
    
    output_shape: 10
    
Example of devices (can be either ``cpu`` or ``gpu``, and both must have the 'name' and 'state' attributes):

    devices:
        - cpu:
            name: "/device:CPU:0"
            state: "Idle"
        
        - gpu:
            name: "/device:GPU:0"
            state: "Train"
        
        - gpu:
            name: "/device:GPU:1"
            state: "Train"
            

Example of layers (they must have the 'name', 'shape' and 'options' attributes; where, 'name' is a string, 'shape' is a list, and 'options' is an attribute containing a list of 'option' attributes with 'name' and 'value' sub-attributes:

      layers:
        - layer:
            name: "Conv2D"
            shape:
              - 32
              - 3
            options:
              - option:
                  name: "padding"
                  value: "same"
        
        - layer:
            name: "Flatten"
            shape:
            options:
            
        - layer:
            name: "Dense"
            shape:
              - 64
            options:

In [12]:
yaml_path = './compilers/example_cnn.yaml'
my_compiler = BaseNetCompiler.build_from_yaml(yaml_path, verbose=True)
my_compiler.layers

[{'Conv2D': ((32, 3), {'padding': 'same'})},
 {'MaxPooling2D': ((), {})},
 {'Conv2D': ((32, 3), {})},
 {'MaxPooling2D': ((), {})},
 {'Flatten': ((), {})},
 {'Dense': ((256,), {})}]

#### 2.4. Compiling the model.

After choosing your method to build your model, and you check it is a valid model, you can use the ``compile()`` method to produce a BaseNetModel. To learn to use the BaseNetModel you can check the advanced guide for BaseNetModel in this Jupyter tutorial.

**Note** that having a valid compiler does not mean that you have a valid model. The model can have compile errors as the provided information is not coherent or there are missing arguments.

In [13]:
print('Is my compiler valid?:', my_compiler.is_valid)
my_compiler.compile()

Is my compiler valid?: True
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


Model object with the following parameters:
Compiler: Compiler with 6 layers, options:
{'loss': 'mean_squared_error', 'optimizer': 'adam'}
Summary: Model: "cnn_from_yaml"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 256, 256, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 128, 128, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 126, 126, 32)      9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 63, 63, 32)       0         
 2D)                                 

#### 3. Save and load compilers.

To save and load compilers you can use the ``save()`` and ``BaseNetCompiler.load()`` methods. The input arguments for both method is the path.

In [14]:
save_load_path = './example_compiler'
my_compiler.save(save_load_path)
other_compiler = BaseNetCompiler.load(save_load_path)
other_compiler.layers == my_compiler.layers

True

#### 3.1. Export the compiler to a YAML file.

You can also export the model to a YAML file, that can be read and imported:

yaml_path = './example_compiler.yaml'
my_compiler.export(yaml_path)
other_compiler.build_from_yaml(yaml_path)
other_compiler.layers
other_compiler.compile()

#### 4. Setting up the computational devices.

This API uses Tensorflow at a lower level. Tensorflow makes use of the GPUs, however, if the available RAM is not enought, the API can collapse. The method ``BaseNetCompiler.set_up_devices()`` is called every time you initialize the API and open-closes the vision between Tensorflow and the GPUs. The policy to close the vision or open it is looking into the VRAM of the GPU and the available RAM. The API detects the free RAM and does not let that the total VRAM be greater than the available RAM. The formula used is:

    total_VRAM = alpha * available_RAM
    
Where alpha is a number between 0 and 1. At the begining, it is 0.8, so the system will have the 20% of the RAM available, while the other 80% will be used by the GPUs; all this as a default configuration. If you want to use more VRAM or let more RAM free, you can use the ``BaseNetCompiler.set_up_devices()`` method, which parameter is the free RAM percentage you will let available for your machine.

For example, if we want to use 60% of our RAM in training our model and 40% for our machine tasks (dashboards, applications, protocols...) we will run the following line of code:

In [15]:
free_ram = 0.4
BaseNetCompiler.set_up_devices(let_free_ram=free_ram)

Once run, the vision between Tensorflow and the GPUs will be configured to use the 60% of your RAM. 

**Note**: Do not exceed the 90% of the RAM, as it can cause memory problems on your machine. This problems can be solved shutting down the machine, but they are annoying and can collapse during the training process.

#### 5. Visualize the available options (losses, optimizers and layers).

You have all the available layers in the variable ``KERAS_LIST_LAYERS``, the loss functions in ``KERAS_LOSSES`` and the optimizers in ``KERAS_OPTIMIZERS``.

You can look into the Keras documentation by accessing ``tf.keras.layers.<<layer>>.__help__`` method, for example, let's take a look into the options that ``Conv1D`` has:

In [27]:
print('Available layers:\n', KERAS_LIST_LAYERS)
print('\n\nAvailable loss functions:\n', KERAS_LOSSES)
print('\n\nAvailable optimizers:\n', KERAS_OPTIMIZERS)
print('\n\n')
original_keras_module = tf.keras.layers
help(original_keras_module.Dense)

Available layers:
 ['Dense', 'Activation', 'Embedding', 'Masking', 'Lambda', '', 'Conv1D', 'Conv2D', 'Conv3D', 'SeparableConv1D', 'SeparableConv2D', 'DepthwiseConv2D', 'Conv1DTranspose', 'Conv2DTranspose', 'Conv3DTranspose', '', 'MaxPooling1D', 'MaxPooling2D', 'MaxPooling3D', 'AveragePooling1D', 'AveragePooling2D', 'AveragePooling3D', 'GlobalMaxPooling1D', 'GlobalMaxPooling2D', 'GlobalMaxPooling3D', 'GlobalAveragePooling1D', 'GlobalAveragePooling2D', 'GlobalAveragePooling3D', '', 'LSTM', 'GRU', 'SimpleRNN', 'RNN', 'TimeDistributed', 'Bidirectional', 'ConvLSTM1D', 'ConvLSTM2D', 'ConvLSTM3D', '', 'TextVectorization', 'Normalization', 'Discretization', 'CategoryEncoding', 'Hashing', 'StringLookup', 'IntegerLookup', 'Resizing', 'Rescaling', 'CenterCrop', 'RandomCrop', 'RandomFlip', 'RandomTranslation', 'RandomRotation', 'RandomZoom', 'RandomHeight', 'RandomWidth', 'RandomContrast', 'RandomBrightness', '', 'Dropout', 'SpatialDropout1D', 'SpatialDropout2D', 'SpatialDropout3D', 'GaussianDropo

You reached the end of the advanced BaseNetCompiler tutorial. Crongratulations!